<a href="https://colab.research.google.com/github/AntonZUZZ/test/blob/main/Pesochek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TODO:
# канибализм
# эффекты от наук
# эффекты от религий (перекаты из религий, культы и прочее)
# нац идеи

# дальние преспективы TODO:
# Gemini для создания персональных ивентов

# Настройки

Здесь помещены глобальные переменные, которые настраивают работу скрипта.
Желательно ознакомится с ними

In [2]:
# Имя файла со статкой
# Существующий файл заменится на новый после работы скрипта
# Если файла не будет, то стату придется вбить руками
NAME_OF_STAT_FILE = "nation_test.json"

# Касты

In [3]:
# Список каст. При добавлении новой касты в игру, ее нужно добавить в этот список
CAST_FARMERS = "фермеры"
CAST_WORKERS = "рабочие"
CAST_SOLDIERS = "солдаты"
CAST_COMMONERS = "обыватели"
CAST_TOWNFOLK = "горожане"
CAST_INTELLECTUALS = "интеллигенция"
CAST_OFFICIALS = "чиновники"

BASE_CAST_LIST = [CAST_FARMERS, CAST_WORKERS, CAST_SOLDIERS, CAST_COMMONERS, CAST_TOWNFOLK, CAST_INTELLECTUALS, CAST_OFFICIALS]

In [4]:
# Базовая кастовка населения
# Значения распределения для каждой касты из списка BASE_CAST_LIST
# Если BASE_CAST_LIST = ["фермеры", "рабочие", "солдаты",  "обыватели", "горожане", "интеллигенция", "чиновники"]
# А    BASE_CAST_DISTRIBUTION = [0.4, 0.22, 0.14, 0.1, 0.09, 0.05, 0]
# То это означает, что у фермеров 40%, у рабочих 22% и тд
BASE_CAST_DISTRIBUTION_LIST = [0.4, 0.22, 0.14, 0.1, 0.09, 0.05, 0]

# Названия аттрибутов, эффектов и пр

In [5]:
# Названия аттрибутов
ATTR_LOYALTY = "loyalty"
ATTR_POPULATION = "population"
ATTR_SLAVE_POPULATION = "slave_population"

In [6]:
# Названия источеников эффектов
SOURCE_MIGRATION = "Миграция"

In [7]:
# Названия эффектов

# Прирост лояльности задается в дробных числах
SETTING_LOYALTY_GROWTH = "loyalty_growth"

# Рождаемость задается в процентах от популяци
SETTING_BIRTH_RATE = "birth_rate"

# slave_birth_yes - Если есть такая настройка, то рабы работают начинают приростать
SETTING_SLAVE_BIRTH_YES = "slave_birth_yes"

# Миграция оказывает следующие эффекты на нацию:
# Каждый пункт миграции даёт или отнимает рождаемость у поселения(1=1).
# Каждый пункт отрицательной миграции даёт +2 прод. Обывателям и -3 приоритета.
# Каждый пункт положительной миграции даёт -1 лояльность в ход(если у расы есть угнетение меньшинств, то -2 лояльности в ход), -1 ассимиляцию и +3 приоритета.
SETTING_MIGRATION = "migration"

# TODO
SETTING_PRIORITY = "priority"

# TODO
SETTING_PROD_COMMONER = "prod.commoner"

# Эффекты

In [8]:
DEFAULT_EFFECTS = [

    # Эффекты состоят из трех компонентов:
    #   "source" - текстовое описание источника,
    #   "setting" - значение эффекта,
    #   "value" - числовое значение, используемое для применения эффекта (если не нужно, то None)
    #   "duration" - целое число, отражающее длительность. Отрицательное означает перманентный эффект.

    ### Дефолтные приросты
    {"source": "Базовая лояльность", "setting": SETTING_LOYALTY_GROWTH, "value": 0, "duration": -1},
    {"source": "Базовая рождаемость", "setting": SETTING_BIRTH_RATE, "value": 10, "duration": -1},
    {"source": "Базовая миграция", "setting": SETTING_MIGRATION, "value": 1, "duration": -1},

    ### Эффекты наук
    # {"source": "Наука о рабах", "setting": SETTING_SLAVE_BIRTH_YES, "value": None, "duration": -1}
]

# Структура

In [9]:
import math

In [10]:
# Функция для генерации словаря базового распределения по кастам
def generate_base_cast_distribution(cast_list, cast_values):
    if len(cast_list) != len(cast_values):
        raise ValueError(f"Длины списков не совпадают: {len(cast_list)} (касты) vs {len(cast_values)} (значения)")
    return dict(zip(cast_list, cast_values))
BASE_CAST_DISTRIBUTION = generate_base_cast_distribution(BASE_CAST_LIST, BASE_CAST_DISTRIBUTION_LIST)

In [11]:
class Nation:
    def __init__(self, population, slave_population, loyalty, priority, race, religion):
        if not isinstance(population, dict):
            raise ValueError("ERROR: Population must be provided as a dictionary of casts")
        self.race = race
        self.religion = religion
        self.effects = DEFAULT_EFFECTS.copy()
        self.attributes = {
            ATTR_POPULATION: {"value": population, "growth": 0},
            ATTR_SLAVE_POPULATION: {"value": slave_population, "growth": 0},
            ATTR_LOYALTY: {"value": loyalty, "growth": 0},
            "priority": {cast: 0 for cast in BASE_CAST_DISTRIBUTION},
            "current_cast_distribution": BASE_CAST_DISTRIBUTION.copy()
        }

    # Проверка - установлен эффект
    def has_passive_effect(self, setting):
        return any(effect["setting"] == setting for effect in self.effects)
    # Подсчет значения эффекта
    def calculate_effect(self, name):
        return sum(effect['value'] for effect in self.effects if effect['setting'] == name)
    # Добавить пассивный эффект
    def add_passive_effect(self, source, setting, value):
        self.effects.append({"source": source, "setting": setting, "value": value})
        self.calculate_attributes_growth()
    # Удалить эффекты источника
    def remove_effects_by_source(self, source):
        self.effects = [effect for effect in self.effects if effect["source"] != source]
        self.calculate_attributes_growth()

    # Рассчет приростов аттрибутов
    def calculate_attributes_growth(self):
        self.migration_effect_update()

        for attr in self.attributes:
            if attr == ATTR_LOYALTY:
                self.attributes[attr]['growth'] = self.calculate_effect(SETTING_LOYALTY_GROWTH)
            if attr == ATTR_POPULATION:
                self.attributes[attr]['growth'] = self.get_total_population() * (self.calculate_effect(SETTING_BIRTH_RATE) / 100)
            if attr == ATTR_SLAVE_POPULATION:
                if (self.has_passive_effect(SETTING_SLAVE_BIRTH_YES)):
                    self.attributes[attr]['growth'] = self.attributes[attr]['value'] * (self.calculate_effect(SETTING_BIRTH_RATE) / 100)
                else:
                    self.attributes[attr]['growth'] = 0

    # Влияние текущей миграции
    def migration_effect_update(self):
        self.remove_effects_by_source(SOURCE_MIGRATION)
        migration = self.calculate_effect(SETTING_MIGRATION)

        # Каждый пункт миграции даёт или отнимает рождаемость у поселения(1=1).
        self.add_passive_effect(SOURCE_MIGRATION, SETTING_BIRTH_RATE, migration)

        # Каждый пункт отрицательной миграции даёт +2 прод. Обывателям и -3 приоритета.
        if (migration < 0):
            self.add_passive_effect(SOURCE_MIGRATION, SETTING_PRIORITY, -3 * abs(migration))
            self.add_passive_effect(SOURCE_MIGRATION, SETTING_PROD_COMMONER, 2 * abs(migration))

        # Каждый пункт положительной миграции даёт -1 лояльность в ход(если у расы есть угнетение меньшинств, то -2 лояльности в ход), -1 ассимиляцию и +3 приоритета.
        if (migration["birth_rate"] > 0):
            #TODO "угнетение меньшинств"
            #TODO
            print("TODO")

    # Суммарное население всех каст
    def get_total_population(self):
          return sum(self.attributes[ATTR_POPULATION]['value'].values())

    # Добавить население рабов
    def add_slave_population(self, new_slave):
        self.attributes[ATTR_SLAVE_POPULATION]['value'] += new_slave

    # Добавить население кастовое
    def add_population(self, count):
        initial_population = self.get_total_population()

        # Распределение нового населения по текущей кастовке
        for caste in self.attributes['current_cast_distribution']:
            caste_percentage = self.attributes['current_cast_distribution'][caste]
            additional_population = int(count * caste_percentage)
            self.attributes[ATTR_POPULATION]['value'][caste] += additional_population

        # Корректировка отрицательного прироста населения
        while any(pop < 0 for pop in self.attributes[ATTR_POPULATION]['value'].values()):
            for caste in self.attributes[ATTR_POPULATION]['value']:
                if self.attributes[ATTR_POPULATION]['value'][caste] < 0:
                    negative_population = self.attributes[ATTR_POPULATION]['value'][caste]
                    self.attributes[ATTR_POPULATION]['value'][caste] = 0
                    positive_castes = [c for c in self.attributes[ATTR_POPULATION]['value'] if c != caste and self.attributes[ATTR_POPULATION]['value'][c] > 0]
                    negative_growth_per_caste = math.ceil(abs(negative_population) / len(positive_castes))
                    for rc in positive_castes:
                        self.attributes[ATTR_POPULATION]['value'][rc] = self.attributes[ATTR_POPULATION]['value'][rc] - negative_growth_per_caste

        # Корректировка для устранения несоответствия в общем приросте населения.
        # Остатки равномерно (буквально по единичке) распределяем между всеми ненулевыми кастами
        # Это нужный костыль, ибо из-за округлений может происходить всякая хрень
        while sum(self.attributes[ATTR_POPULATION]['value'].values()) != initial_population + count:
            for caste in positive_castes:
                if sum(self.attributes[ATTR_POPULATION]['value'].values()) == initial_population + count:
                    break
                self.attributes[ATTR_POPULATION]['value'][caste] += 1

    def apply_priority(self):
        total_priority = sum(self.attributes["priority"].values())

        for cast in BASE_CAST_DISTRIBUTION:
            base_value = BASE_CAST_DISTRIBUTION[cast]
            priority_bonus = self.attributes["priority"][cast]
            self.attributes["current_cast_distribution"][cast] = base_value * (1 - total_priority) + priority_bonus

    # Выводим в строку информацию о нашей нации.
    def __str__(self):
        # Формирование строки для населения по кастам
        population_str = "\n".join([f"{cast}: {self.attributes[ATTR_POPULATION]['value'][cast]}" for cast in self.attributes[ATTR_POPULATION]['value']])
        # Формирование строки для рабов
        slave_population_str = "\n".join([f"{self.attributes[ATTR_SLAVE_POPULATION]['value']}"])
        # Формирование строки для приоритетов
        priority_str = "\n".join([f"{cast}: {self.attributes['priority'][cast]*100}%" for cast in self.attributes['priority']])
        # Формирование строки для текущей кастовки
        current_cast_dist_str = "\n".join([f"{cast}: {self.attributes['current_cast_distribution'][cast]*100}%" for cast in self.attributes['current_cast_distribution']])
        # Эффекты нации
        effects_str = "\nЭффекты:\n"
        for e in self.effects:
            source = e['source']
            setting = e['setting']
            value = e['value']
            effects_str += f"\"{source}\" дает эффект \"{setting}\" в значении {value}\n"

        return (f"Информация о нации:\n"
                f"Раса: {self.race}\n"
                f"Религия: {self.religion}\n"
                f"Общее население: {self.get_total_population()}\n"
                f"Лояльность: {self.attributes[ATTR_LOYALTY]['value']} (+{self.attributes[ATTR_LOYALTY]['growth']})\n"
                f"Население (по кастам):\n{population_str}\n"
                f"Рабы: {slave_population_str}\n"
                f"Текущие приоритеты (в %):\n{priority_str}\n"
                f"Текущая кастовка (в %):\n{current_cast_dist_str}\n"
                f"\n{effects_str}\n")


In [12]:
#Функция создания дефолтной нации (только для тестирования)
def create_nation():
    # while True:
    #     try:
    #         population = int(input("Enter population: "))
    #         loyalty = int(input("Enter loyalty: "))
    #         birth_rate = float(input("Enter birth rate: "))
    #         break
    #     except ValueError:
    #         print("Invalid input. Please enter numeric values for population, loyalty, and birth rate.")

    # race = input("Enter race: ")
    # religion = input("Enter religion: ")

    # return Nation(population, loyalty, birth_rate, race, religion)

    # Генерация начального населения для каждой касты
    initial_population = {cast: 10000 for cast in BASE_CAST_LIST}
    # Создание объекта нации
    my_nation = Nation(
        population=initial_population,
        slave_population = 1000,
        loyalty=50,
        priority={cast: 0 for cast in BASE_CAST_DISTRIBUTION},
        race="Human",
        religion="Polytheism"
    )
    return my_nation

# Утилитатрное

In [13]:
import json

# функции чтения и записи статистики
def read_stats(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return Nation(**data)
    except FileNotFoundError:
        print("File not found.")
        return None

def save_stats(settlement, filename):
    with open(filename, 'w') as file:
        json.dump(settlement.__dict__, file)

# Функции пользователя

In [14]:
    # print()
    # print("Данные о песочной нации получены!")
    # print("Приступаем к действиям!")

    # while True:
    #     action = input("Enter an NOT permanent action (\"exit\" to exit): ")
    #     if action == "exit":
    #         break
    #     nation.apply_not_permanent_action(action)

    # nation.update()
    # save_stats(nation, NAME_OF_STAT_FILE)

In [15]:
# Функция получения статки Нации
def get_nation():
    nation = read_stats(NAME_OF_STAT_FILE)
    if nation is None:
        nation = create_nation()
    return nation

In [16]:
# Функция для установки приоритета для касты
#TODO: Нужно выставлять приоритет у всех каст разом. Нельзя отрицательный
def set_priority(nation, cast, priority_value):
    if cast not in nation.attributes["priority"]:
        raise ValueError(f"Неверное название касты: {cast}")
    #TODO: Проверка что он может такой приоритет выставить
    nation.attributes["priority"][cast] = priority_value
    nation.apply_priority()

In [17]:
# Функция сохранения нации в файл
def save_nation(nation):
    save_stats(nation, NAME_OF_STAT_FILE)

In [18]:
# Функция прибавки населения за ход
def grow_population(nation):
    nation.add_population(nation.attributes['population']['growth'])
    nation.add_slave_population(nation.attributes['slave_population']['growth'])

In [19]:
# Обновление статки. Совершение хода.
def update(nation):
    nation.calculate_attributes_growth()

    # прирост населения в этом ходу
    grow_population(nation)

# Основной функционал с примерами

In [20]:
nation = get_nation()

File not found.


In [21]:
# Просмотр статки
print(nation)

Информация о нации:
Раса: Human
Религия: Polytheism
Общее население: 70000
Лояльность: 50 (+0)
Население (по кастам):
фермеры: 10000
рабочие: 10000
солдаты: 10000
обыватели: 10000
горожане: 10000
интеллигенция: 10000
чиновники: 10000
Рабы: 1000
Текущие приоритеты (в %):
фермеры: 0%
рабочие: 0%
солдаты: 0%
обыватели: 0%
горожане: 0%
интеллигенция: 0%
чиновники: 0%
Текущая кастовка (в %):
фермеры: 40.0%
рабочие: 22.0%
солдаты: 14.000000000000002%
обыватели: 10.0%
горожане: 9.0%
интеллигенция: 5.0%
чиновники: 0%
:

Эффекты:
"Базовая лояльность" дает эффект "loyalty_growth" в значении 0
"Базовая рождаемость" дает эффект "birth_rate" в значении 10
"Базовая миграция" дает эффект "migration" в значении 1




In [22]:
# Установка приоритета и просмотр статки
set_priority(nation, "фермеры", 0.4)
set_priority(nation, "солдаты", -0.4)
print(nation)

Информация о нации:
Раса: Human
Религия: Polytheism
Общее население: 70000
Лояльность: 50 (+0)
Население (по кастам):
фермеры: 10000
рабочие: 10000
солдаты: 10000
обыватели: 10000
горожане: 10000
интеллигенция: 10000
чиновники: 10000
Рабы: 1000
Текущие приоритеты (в %):
фермеры: 40.0%
рабочие: 0%
солдаты: -40.0%
обыватели: 0%
горожане: 0%
интеллигенция: 0%
чиновники: 0%
Текущая кастовка (в %):
фермеры: 80.0%
рабочие: 22.0%
солдаты: -26.0%
обыватели: 10.0%
горожане: 9.0%
интеллигенция: 5.0%
чиновники: 0.0%
:

Эффекты:
"Базовая лояльность" дает эффект "loyalty_growth" в значении 0
"Базовая рождаемость" дает эффект "birth_rate" в значении 10
"Базовая миграция" дает эффект "migration" в значении 1




In [23]:
grow_population(nation)
print(nation)

Информация о нации:
Раса: Human
Религия: Polytheism
Общее население: 70000
Лояльность: 50 (+0)
Население (по кастам):
фермеры: 10000
рабочие: 10000
солдаты: 10000
обыватели: 10000
горожане: 10000
интеллигенция: 10000
чиновники: 10000
Рабы: 1000
Текущие приоритеты (в %):
фермеры: 40.0%
рабочие: 0%
солдаты: -40.0%
обыватели: 0%
горожане: 0%
интеллигенция: 0%
чиновники: 0%
Текущая кастовка (в %):
фермеры: 80.0%
рабочие: 22.0%
солдаты: -26.0%
обыватели: 10.0%
горожане: 9.0%
интеллигенция: 5.0%
чиновники: 0.0%
:

Эффекты:
"Базовая лояльность" дает эффект "loyalty_growth" в значении 0
"Базовая рождаемость" дает эффект "birth_rate" в значении 10
"Базовая миграция" дает эффект "migration" в значении 1




In [24]:
# Сохранение статки в файл
# save_nation(nation)